### Activations, Gradients and BatchNorm
We have build an MLP in part2 and now we need to understand how the activations and gradients flow in a Neural Network because the othe more powerful architectures used nowadays are very powerful model in learning any function in principle but they are very hard to train because of problems with activations and gradients.

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff

In [2]:
with open("D://Datasets/names.txt", 'r') as file:
    names = file.read().splitlines()
names[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
vocabulary = sorted(list(set(''.join(names))))
chartoidx = {}
idxtochar = {}
chartoidx['.'] = 0   # Putting a special token to denote the start and the end of a sentence.
idxtochar[0] = '.'
for i,char in enumerate(vocabulary):
    chartoidx[char] = i+1
    idxtochar[i+1] = char

chartoidx

{'.': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [4]:
block_size = 3 # This would be used to set the number of context letters that would be in the word.
# blank_context = idxtochar[0] * block_size  # This would produce the context consisting of only special character "."
# print(f"The blank context is {blank_context}")

# Now we would need to split the names into train, dev and test sets
train_names = names[:int(0.8*len(names))]
dev_names = names[int(0.8*len(names)): int(0.9*len(names))]
test_names = names[int(0.9*len(names)):]

# Now we need to define our xs and ys
X_train, y_train = [],[]
X_dev, y_dev = [],[]
X_test, y_test = [], []

# Now we need to add the data into our xs and ys:
for i,split in enumerate([train_names, dev_names, test_names]):
    for word in split:
        word = word+"."
        blank_context = [0] * block_size  # This would produce the context consisting of only special character "."
        # xs.append(blank_context)
        if i == 0:
            for ch in word:
                X_train.append(blank_context)
                y_train.append(chartoidx[ch])
                blank_context = blank_context[1:] + [chartoidx[ch]]
        if i == 1:
            for ch in word:
                X_dev.append(blank_context)
                y_dev.append(chartoidx[ch])
                blank_context = blank_context[1:] + [chartoidx[ch]]
        if i == 2:
            for ch in word:
                X_test.append(blank_context)
                y_test.append(chartoidx[ch])
                blank_context = blank_context[1:] + [chartoidx[ch]]
      
        # print(blank_context)
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_dev = torch.tensor(X_dev)
y_dev = torch.tensor(y_dev)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)  
print(len(X_train), len(y_train))
print(len(X_dev), len(y_dev))
print(len(X_test), len(y_test))

182778 182778
22633 22633
22735 22735


In [5]:
X_train[3]

tensor([ 5, 13, 13])

In [6]:
print(len(vocabulary))

26


In [7]:
# lets consolidate the full code
block_size = 3
embedding_size = 10
hidden_neurons = 200

g = torch.Generator().manual_seed(2147483647) # Declare the generator with the manual seed.

c = torch.randn((len(chartoidx),embedding_size), generator=g)   # We are using 2 dimensional embeddings
print(c.shape)

w1 = torch.randn((block_size*embedding_size,hidden_neurons), generator=g) # We are  using hidden layers with 100 neurons

b1 = torch.randn((hidden_neurons), generator=g)   

w2 = torch.rand((hidden_neurons,len(chartoidx)), generator=g) # The output layer contains 27 neurons 

b2 = torch.rand((len(chartoidx)), generator=g)

parameters = [c,w1,b1,w2,b2]

p = sum([x.nelement() for x in parameters])
for p in parameters:
    p.requires_grad = True
p

torch.Size([27, 10])


tensor([0.7873, 0.8429, 0.8973, 0.9537, 0.6088, 0.0610, 0.4129, 0.4254, 0.8633,
        0.3893, 0.6857, 0.2104, 0.5713, 0.3860, 0.4604, 0.5812, 0.8003, 0.6140,
        0.9329, 0.0821, 0.9237, 0.0694, 0.6295, 0.5813, 0.2378, 0.9188, 0.9077],
       requires_grad=True)

In [8]:
# Now lets do the training looop
# Forward pass
data_dict ={'train':[X_train, y_train], 
            'dev':[X_dev, y_dev],
            'test':[X_test, y_test]}
lri = []
lossi = []
epochs = 100000
def training_loop(epochs, data_dict, is_val_set = False):
    if is_val_set == False:
        for i in range(epochs):
            ix = torch.randint(0, data_dict['train'][0].shape[0], (32,))  # this would generate 32 random indexes which we can filter our dataset to train over a mini-batch
            embed = c[data_dict['train'][0][ix]]   # (32,3,2)
            
            h1 = torch.tanh(embed.view(-1,block_size*embedding_size) @ w1 +b1)
            logits = h1 @ w2 + b2
            loss = F.cross_entropy(logits, data_dict['train'][1][ix])
            # print(loss.item())
            # Zero out the gradients
            for param in parameters:
                param.grad = None
                
            # Backward Pass
            loss.backward()


            # Update the parameters
            lr = 0.1 if i<50000 else 0.01
            for p in parameters:
                p.data += -lr*p.grad

            # # track the loss stats
            # lri.append(lre[i])
            # lossi.append(loss.item())
            if i % 10000==0:
                print(f"The training loss is : {loss.item()}")
    else:
        with torch.no_grad():
            for i in range(epochs):
                val_ix = torch.randint(0, data_dict['dev'][0].shape[0], (32,))  # this would generate 32 random indexes which we can filter our dataset to train over a mini-batch
                val_embed = c[data_dict['dev'][0][val_ix]]   # (32,3,2)
                
                h1 = torch.tanh(val_embed.view(-1,block_size*embedding_size) @ w1 +b1)
                val_logits = h1 @ w2 + b2
                val_loss = F.cross_entropy(val_logits, data_dict['dev'][1][val_ix])
            print(f"The Validation loss is :{val_loss}")

training_loop(epochs, data_dict, is_val_set=False)

The training loss is : 7.503610134124756
The training loss is : 2.4590415954589844
The training loss is : 2.4872210025787354
The training loss is : 2.456273317337036
The training loss is : 2.223405122756958
The training loss is : 2.4477710723876953
The training loss is : 2.133476495742798
The training loss is : 1.9146534204483032
The training loss is : 2.038461923599243
The training loss is : 2.0174314975738525


In [9]:
training_loop(epochs, data_dict, is_val_set=True)

The Validation loss is :2.2140512466430664


Now the first problem that we see that the our initialization is not very good becuase some letters are having a high probability from the start. Ideally all the characters in the vocabulary at the start should have equal probabilities. So if all the characters have equal probabilities the loss should be   
```python
        loss = -torch.tensor(1/27.0).log()
        loss = 3.2958
```
So ideally the starting loss should be 3.2958, but as we can see that the starting loss is 7.83 so this is a problem because what happens is that the Network initialization gives up imbalanced weigths to some characters so if by chance the weights are predefined in such a way that the output is correct then the loss would be very low, which would indicate that the Network has learned nicely, but in reality it was just a chance, conversly if the weights are imabalanced in such a way that the incorrect character gets outputted then there is a problem that the loss would be very high and there is also a chance that the NN will show that the network has not learnt at all even though it has learnt some things.

In [58]:
-torch.tensor(1/27.0).log()

tensor(3.2958)

Another problem is that the tanh function gets sturates due to activation and does not backpropagates the gradients easily resulting in dead neurons.\
We can combat this by multiplying our weights and biases initialization with a very small number.

In [15]:
# Lets test the training and validation losses after multiplying the paramters by a small number to mitigate the losses.
# lets consolidate the full code
block_size = 3
embedding_size = 10
hidden_neurons = 200

g = torch.Generator().manual_seed(2147483647) # Declare the generator with the manual seed.

c = torch.randn((len(chartoidx),embedding_size), generator=g)   # We are using 2 dimensional embeddings
print(c.shape)

w1 = torch.randn((block_size*embedding_size,hidden_neurons), generator=g) * 0.2 # We are  using hidden layers with 100 neurons

b1 = torch.randn((hidden_neurons), generator=g)   * 0.01

w2 = torch.rand((hidden_neurons,len(chartoidx)), generator=g) * 0.01 # The output layer contains 27 neurons 

b2 = torch.rand((len(chartoidx)), generator=g) * 0.01

parameters = [c,w1,b1,w2,b2]

p = sum([x.nelement() for x in parameters])
for p in parameters:
    p.requires_grad = True

torch.Size([27, 10])


In [16]:
# Now lets train and see if it had some impact
# Now lets do the training looop
# Forward pass
data_dict ={'train':[X_train, y_train], 
            'dev':[X_dev, y_dev],
            'test':[X_test, y_test]}
lri = []
lossi = []
epochs = 100000
def training_loop(epochs, data_dict, is_val_set = False):
    if is_val_set == False:
        for i in range(epochs):
            ix = torch.randint(0, data_dict['train'][0].shape[0], (32,))  # this would generate 32 random indexes which we can filter our dataset to train over a mini-batch
            embed = c[data_dict['train'][0][ix]]   # (32,3,2)
            
            h1 = torch.tanh(embed.view(-1,block_size*embedding_size) @ w1 +b1)
            logits = h1 @ w2 + b2
            loss = F.cross_entropy(logits, data_dict['train'][1][ix])
            # print(loss.item())
            # Zero out the gradients
            for param in parameters:
                param.grad = None
                
            # Backward Pass
            loss.backward()


            # Update the parameters
            lr = 0.1 if i<50000 else 0.01
            for p in parameters:
                p.data += -lr*p.grad

            # # track the loss stats
            # lri.append(lre[i])
            # lossi.append(loss.item())
            if i % 10000==0:
                print(f"The training loss is : {loss.item()}")
    else:
        with torch.no_grad():
            for i in range(epochs):
                val_ix = torch.randint(0, data_dict['dev'][0].shape[0], (32,))  # this would generate 32 random indexes which we can filter our dataset to train over a mini-batch
                val_embed = c[data_dict['dev'][0][val_ix]]   # (32,3,2)
                
                h1 = torch.tanh(val_embed.view(-1,block_size*embedding_size) @ w1 +b1)
                val_logits = h1 @ w2 + b2
                val_loss = F.cross_entropy(val_logits, data_dict['dev'][1][val_ix])
            print(f"The Validation loss is :{val_loss}")

training_loop(epochs, data_dict, is_val_set=False)

The training loss is : 3.2970218658447266
The training loss is : 2.2902276515960693
The training loss is : 1.8809808492660522
The training loss is : 2.0626513957977295
The training loss is : 2.1294167041778564
The training loss is : 1.8811578750610352
The training loss is : 2.1255714893341064
The training loss is : 1.9401602745056152
The training loss is : 2.0361762046813965
The training loss is : 1.7744632959365845


In [17]:
training_loop(epochs, data_dict, is_val_set=True)

The Validation loss is :2.0560131072998047


As we can see that we converged to a better training loss and validation loss and the training and validation losses are pretty close. This is indicates that the model has definetly learned something.

Here for the above problems we were dealing with very shallow network with only one layer, so inspite of bad initialization of weights the Neural Network was able to learn and we were able to get a low loss. But as we move to deeper networks with more layers this will not work and it becomes a problem.

But now we have made up numbers of 0.2, 0.1, 0.01 that we have multiplied our weights and biases with to achieve better initialization. But how do we come up with these numbers that is a question that bugs me.

So this initialization is not just a random number. It is calculated by something known as kaiming normal which initializes the weights as having zero mean and unit standard_deviation. Another way to make it of unit Standard Deviation is to divide the weights by the square_root of the number of inputs of the weights also known as the `fan_in`.

torch has a
```python
    torch.nn.init.kaiming_normal(tensor, a = 0, mode = 'fan_in', non_linearity = 'relu')
```
which initializes the weigths of zero mean and unit standard deviation.\
The mode parameter takes two values `fan_in` and `fan_out` fan_in transforms the input weights and fan_out initializes the gradients to have zero mean and unit std. But as initializeing the initial weights also initializes the gradients approximately we just leave it as fan_in.

Recent advances in normalization techniques like batchnorm and newer optimizers like adam and rmsprop have made it possible to train very deep neural networks without thinking about initializations too much. So we would not focus too much on them.

To initialize our weights in the form of kaimin_normal we would multiply the weights by gain_of_activation_function/(fan_in)**0.5, which comes out to be
(5/3) / 30**0.5 = 0.3

In [25]:
# Lets test the training and validation losses after multiplying the paramters by a small number to mitigate the losses.
# lets consolidate the full code
block_size = 3
embedding_size = 10
hidden_neurons = 200

g = torch.Generator().manual_seed(2147483647) # Declare the generator with the manual seed.

c = torch.randn((len(chartoidx),embedding_size), generator=g)   # We are using 2 dimensional embeddings
print(c.shape)

# Initializing w1 by the kaiming normal initialization
w1 = torch.randn((block_size*embedding_size,hidden_neurons), generator=g) * ((5/3) / ((embedding_size*block_size)**0.5))   #* 0.2 # We are  using hidden layers with 100 neurons

b1 = torch.randn((hidden_neurons), generator=g)   * 0.01

w2 = torch.rand((hidden_neurons,len(chartoidx)), generator=g) * 0.01 # The output layer contains 27 neurons 

b2 = torch.rand((len(chartoidx)), generator=g) * 0.01

parameters = [c,w1,b1,w2,b2]

p = sum([x.nelement() for x in parameters])
for p in parameters:
    p.requires_grad = True

torch.Size([27, 10])


In [26]:
data_dict ={'train':[X_train, y_train], 
            'dev':[X_dev, y_dev],
            'test':[X_test, y_test]}
lri = []
lossi = []
epochs = 100000
def training_loop(epochs, data_dict, is_val_set = False):
    if is_val_set == False:
        for i in range(epochs):
            ix = torch.randint(0, data_dict['train'][0].shape[0], (32,))  # this would generate 32 random indexes which we can filter our dataset to train over a mini-batch
            embed = c[data_dict['train'][0][ix]]   # (32,3,2)
            
            h1 = torch.tanh(embed.view(-1,block_size*embedding_size) @ w1 +b1)
            logits = h1 @ w2 + b2
            loss = F.cross_entropy(logits, data_dict['train'][1][ix])
            # print(loss.item())
            # Zero out the gradients
            for param in parameters:
                param.grad = None
                
            # Backward Pass
            loss.backward()


            # Update the parameters
            lr = 0.1 if i<50000 else 0.01
            for p in parameters:
                p.data += -lr*p.grad

            # # track the loss stats
            # lri.append(lre[i])
            # lossi.append(loss.item())
            if i % 10000==0:
                print(f"The training loss is : {loss.item()}")
    else:
        with torch.no_grad():
            for i in range(epochs):
                val_ix = torch.randint(0, data_dict['dev'][0].shape[0], (32,))  # this would generate 32 random indexes which we can filter our dataset to train over a mini-batch
                val_embed = c[data_dict['dev'][0][val_ix]]   # (32,3,2)
                
                h1 = torch.tanh(val_embed.view(-1,block_size*embedding_size) @ w1 +b1)
                val_logits = h1 @ w2 + b2
                val_loss = F.cross_entropy(val_logits, data_dict['dev'][1][val_ix])
            print(f"The Validation loss is :{val_loss}")

training_loop(epochs, data_dict, is_val_set=False)

The training loss is : 3.2980849742889404
The training loss is : 2.269784688949585
The training loss is : 2.423513174057007
The training loss is : 2.2749626636505127
The training loss is : 2.0059123039245605
The training loss is : 2.538508415222168
The training loss is : 1.7103546857833862
The training loss is : 2.013820171356201
The training loss is : 2.200108289718628
The training loss is : 1.8696880340576172


In [27]:
training_loop(epochs, data_dict, is_val_set=True)

The Validation loss is :2.250772476196289


So the losses are roughly the same but we are putting any magic numbers rather we are using a semi-pricipled initialization strategy which scales very well with deeper networks.\
But the precise setting of the initializations is not important to train NN today because of the recent advances in Neural Networks and on of the innovation is BatchNormalization.

### Batch Normalization
1. So batch normalization is the technique that is used to make the preactivations of a Neural Network to become close to a gaussian distribution in the forward pass of a Neural Network. This is done so to intialize the pre-activation to be in the learning part of the activation function because, lets say if the activation function was a tanh, If the preactivations are close to zero the Neural netork would output close to zero activation to the next layer which would make the training difficult, conversely if it was a bigger number than 1 then it would output 1 and the backpropagation would make the gradients as 0 and the Neural Network won't learn anything as the gradients would be zeored out when it passes through that neuron.
2. So we need to make the preactivation follow a gaussian distribution with a mean of 0 and std of 1 so that both the forward and backward passes are done correctly by the Neural netwrok and it is easier to train(this is very useful in deep NN).
3. We do that by calculating the mean and std of each batch(hence the name BatchNorm) and then we subtract the mean from each individual examples and divide by the std. This makes the layers to adhere to a gaussian distribution in the forward pass and we can easily pass the gradients in backprop. We also multiply the result of subtracting the mean and dividing by the std by a factor known as `scale` and also add another factor known as `shift` to break the symmetry of the gaussian distribution and to make the resulting gaussian bigger or smaller and also to shift it slightly to one side or the other and to train the networks better, because we want the preactivation to be roughly gaussian and not forced to be gaussian.
4. BatchNorm stems from the fact that if we want the activations to be unit gaussian so that they would train better then why not make them roughly to be unit gaussian.
5. WE would add these batchnorm layers throughout the neural and what it does that it controls the scale of the activation throughout the Neural Net.It also helps us in the way that it does not require mathematics and care about the activation distributions manually. It sort of does it automatically.
6. The stability offered by batchNormalization actually comes at a cost:
    1. First we started with individual examples and the activations of the layers were just for those examples. Then we started to do minibatches of examples but the netwrok processed single examples at a time and grouping them in batches was just an efficiency thing.
    2. But now in batchnormalization each single example depends upon other examples also we are coupling these examples mathematically, so the previous layer activations and the logits are not just only the function of individual examples but they are also a function of other examples(as they are influenced by the mean and the std of the batch).
    3. So the h and the logits for each individual example will jitter and we would think that this would be a bug, but this type of jitter is actually beneficial for the training because it acts as a regularizer. SO what is happening is that due to this jitter and because the examples are being generated randomly in the minibatch, at each iteration each example even if it the same example is slightly different because of the mean and std of the other examples(which will always be different in a batch). So the neural network is seeing roughly the same but slightlt different examples. This kind of is a form of data augmentation. This has a regularizing effect.
    4. This property of acting as a regularizer has made us to not stop using BatchNormalization.
    5. Due to the coupling property with other examples some people have tried other methods of normalizing the batch without coupling which has led to techniques such as Layer Normalization, Group Normalization and so on.
    6. It stablizes the training of deep neural networks.
7. Now we have trained the Network, but the how do we feed a single example to this network so that it makes inference, because batchnorm works with batches and we should need batches but for single examples there are no batches. So what we do is that we take the mean and std for the entire dataset and normalize the single example with the mean and std of the entire dataset and then do inference on the example.

Now lets initialize the scale and the shift paramters along with the weights and the biases

In [13]:
# Lets test the training and validation losses after multiplying the paramters by a small number to mitigate the losses.
# lets consolidate the full code
block_size = 3
embedding_size = 10
hidden_neurons = 200

g = torch.Generator().manual_seed(2147483647) # Declare the generator with the manual seed.

c = torch.randn((len(chartoidx),embedding_size), generator=g)   # We are using 2 dimensional embeddings
print(c.shape)

bngain = torch.ones((1,hidden_neurons))
bnbias = torch.zeros((1,hidden_neurons))

# Initializing w1 by the kaiming normal initialization
w1 = torch.randn((block_size*embedding_size,hidden_neurons), generator=g) * ((5/3) / ((embedding_size*block_size)**0.5))   #* 0.2 # We are  using hidden layers with 100 neurons

b1 = torch.randn((hidden_neurons), generator=g)   * 0.01

w2 = torch.rand((hidden_neurons,len(chartoidx)), generator=g) * 0.01 # The output layer contains 27 neurons 

b2 = torch.rand((len(chartoidx)), generator=g) * 0.01

parameters = [c,w1,b1,w2,b2, bngain, bnbias]

p = sum([x.nelement() for x in parameters])
for p in parameters:
    p.requires_grad = True

torch.Size([27, 10])


In [14]:
data_dict ={'train':[X_train, y_train], 
            'dev':[X_dev, y_dev],
            'test':[X_test, y_test]}
lri = []
lossi = []
epochs = 100000
def training_loop(epochs, data_dict, is_val_set = False):
    if is_val_set == False:
        for i in range(epochs):
            ix = torch.randint(0, data_dict['train'][0].shape[0], (32,))  # this would generate 32 random indexes which we can filter our dataset to train over a mini-batch
            embed = c[data_dict['train'][0][ix]]   # (32,3,2)
            
            hpreact = embed.view(-1,block_size*embedding_size) @ w1 +b1
            hpreact = bngain* (hpreact-hpreact.mean(0,keepdim=True)) / hpreact.std(0,keepdim=True) + bnbias  # Batch Normalization Layer
            h1 = torch.tanh(hpreact)
            logits = h1 @ w2 + b2

            loss = F.cross_entropy(logits, data_dict['train'][1][ix])
            # print(loss.item())
            # Zero out the gradients
            for param in parameters:
                param.grad = None
                
            # Backward Pass
            loss.backward()


            # Update the parameters
            lr = 0.1 if i<50000 else 0.01
            for p in parameters:
                p.data += -lr*p.grad

            # # track the loss stats
            # lri.append(lre[i])
            # lossi.append(loss.item())
            if i % 10000==0:
                print(f"The training loss is : {loss.item()}")
    else:
        with torch.no_grad():
            for i in range(epochs):
                val_ix = torch.randint(0, data_dict['dev'][0].shape[0], (32,))  # this would generate 32 random indexes which we can filter our dataset to train over a mini-batch
                val_embed = c[data_dict['dev'][0][val_ix]]   # (32,3,2)

                hpreact = val_embed.view(-1,block_size*embedding_size) @ w1 +b1
                hpreact = bngain* (hpreact-hpreact.mean(0,keepdim=True)) / hpreact.std(0,keepdim=True) + bnbias  # Batch Normalization Layer
                
                h1 = torch.tanh(hpreact)
                val_logits = h1 @ w2 + b2
                val_loss = F.cross_entropy(val_logits, data_dict['dev'][1][val_ix])
            print(f"The Validation loss is :{val_loss}")

training_loop(epochs, data_dict, is_val_set=False)

The training loss is : 3.2935962677001953
The training loss is : 2.284334897994995
The training loss is : 1.9388840198516846
The training loss is : 2.55128812789917
The training loss is : 1.9252368211746216
The training loss is : 1.9436469078063965
The training loss is : 1.8702313899993896
The training loss is : 2.068390369415283
The training loss is : 2.155322790145874
The training loss is : 2.465070962905884


In [21]:
# Calculating the mean and std of the preactivations of the entire dataset
with torch.no_grad():
    # get the embeddings of the entire dataset
    embed = c[data_dict['train'][0]]

    hpreact = embed.view(embed.shape[0], -1) @ w1 + b1
    
    # Calculate the mean and std of the full dataset
    bnmean = hpreact.mean(0,keepdim=True)
    bnstd = hpreact.std(0,keepdim=True)

print(bnmean.shape, bnstd.shape)

torch.Size([1, 200]) torch.Size([1, 200])


In [25]:
# lets infer from one training example
embed = c[data_dict['dev'][0]]

hpreact = embed.view(embed.shape[0], -1) @ w1 + b1
hpreact = bngain*((hpreact - bnmean)/bnstd) + bnbias
h = torch.tanh(hpreact)
logits = h @ w2 + b2
print(logits.shape)
print(data_dict['dev'][1].shape)
loss = F.cross_entropy(logits, data_dict['dev'][1])
print(f"val loss after bn is : {loss}")


torch.Size([22633, 27])
torch.Size([22633])
val loss after bn is : 2.342071294784546


In [15]:
training_loop(epochs, data_dict, is_val_set=True)

The Validation loss is :2.32712984085083


But the problem is that nobody wants to calculate the mean and the std after the training is completed and if the training set is very large it becomes problematic. So what we need to do is we can calculate the bnmean and the bnstd while training. SO what we would do is that we would define dummy mean and dummy std of zeros and ones and as the training progresses we would update those means and std's.\
This calculation and update would be independent of the actual training and backpropagation with no graidents being maintained.

In [41]:
# Lets test the training and validation losses after multiplying the paramters by a small number to mitigate the losses.
# lets consolidate the full code
block_size = 3
embedding_size = 10
hidden_neurons = 200

g = torch.Generator().manual_seed(2147483647) # Declare the generator with the manual seed.

c = torch.randn((len(chartoidx),embedding_size), generator=g)   # We are using 2 dimensional embeddings
print(c.shape)

bngain = torch.ones((1,hidden_neurons))
bnbias = torch.zeros((1,hidden_neurons))

bnmean = torch.zeros((1,hidden_neurons))  # Define the intial mean values 
bnstd = torch.ones((1,hidden_neurons))    # Define the intial std values

# Initializing w1 by the kaiming normal initialization
w1 = torch.randn((block_size*embedding_size,hidden_neurons), generator=g) * ((5/3) / ((embedding_size*block_size)**0.5))   #* 0.2 # We are  using hidden layers with 100 neurons

b1 = torch.randn((hidden_neurons), generator=g)   * 0.01

w2 = torch.rand((hidden_neurons,len(chartoidx)), generator=g) * 0.01 # The output layer contains 27 neurons 

b2 = torch.rand((len(chartoidx)), generator=g) * 0.01

parameters = [c,w1,b1,w2,b2, bngain, bnbias]

p = sum([x.nelement() for x in parameters])
for p in parameters:
    p.requires_grad = True

torch.Size([27, 10])


In [42]:
data_dict ={'train':[X_train, y_train], 
            'dev':[X_dev, y_dev],
            'test':[X_test, y_test]}
lri = []
lossi = []
epochs = 100000
def training_loop(epochs, data_dict, bnmean, bnstd,is_val_set = False):
    if is_val_set == False:
        for i in range(epochs):
            ix = torch.randint(0, data_dict['train'][0].shape[0], (32,))  # this would generate 32 random indexes which we can filter our dataset to train over a mini-batch
            embed = c[data_dict['train'][0][ix]]   # (32,3,2)
            
            hpreact = embed.view(-1,block_size*embedding_size) @ w1 +b1
            
            bnmeani = hpreact.mean(0,keepdim=True)
            bnstdi = hpreact.std(0,keepdim=True)

            with torch.no_grad():
                bnmean = 0.999 * bnmean + 0.001 * bnmeani
                bnstd = 0.999 * bnstd + 0.001 * bnstdi


            hpreact = bngain* ((hpreact - bnmeani) /bnstdi)  + bnbias  # Batch Normalization Layer
            h1 = torch.tanh(hpreact)
            logits = h1 @ w2 + b2

            loss = F.cross_entropy(logits, data_dict['train'][1][ix])
            # print(loss.item())
            # Zero out the gradients
            for param in parameters:
                param.grad = None
                
            # Backward Pass
            loss.backward()


            # Update the parameters
            lr = 0.1 if i<50000 else 0.01
            for p in parameters:
                p.data += -lr*p.grad

            # # track the loss stats
            # lri.append(lre[i])
            # lossi.append(loss.item())
            if i % 10000==0:
                print(f"The training loss is : {loss.item()}")
    else:
        with torch.no_grad():
            for i in range(epochs):
                val_ix = torch.randint(0, data_dict['dev'][0].shape[0], (32,))  # this would generate 32 random indexes which we can filter our dataset to train over a mini-batch
                val_embed = c[data_dict['dev'][0][val_ix]]   # (32,3,2)

                hpreact = val_embed.view(-1,block_size*embedding_size) @ w1 +b1
                hpreact = bngain* (hpreact-hpreact.mean(0,keepdim=True)) / hpreact.std(0,keepdim=True) + bnbias  # Batch Normalization Layer
                
                h1 = torch.tanh(hpreact)
                val_logits = h1 @ w2 + b2
                val_loss = F.cross_entropy(val_logits, data_dict['dev'][1][val_ix])
            print(f"The Validation loss is :{val_loss}")
    
    return bnmean, bnstd

bnmean_running, bnstd_running = training_loop(epochs, data_dict, is_val_set=False, bnmean=bnmean, bnstd=bnstd)

The training loss is : 3.2953152656555176
The training loss is : 2.3873021602630615
The training loss is : 1.996972680091858
The training loss is : 2.0783636569976807
The training loss is : 2.295694351196289
The training loss is : 2.478271245956421
The training loss is : 2.1675498485565186
The training loss is : 1.7862409353256226
The training loss is : 2.4602460861206055
The training loss is : 2.404066562652588


Then we would use the bnmeand and bnstd calculated through the training process to do calculations at inference time.


In [43]:
print(bnmean.shape, bnstd.shape)

torch.Size([1, 200]) torch.Size([1, 200])


In [45]:
# lets infer from one training example
embed = c[data_dict['dev'][0]]

hpreact = embed.view(embed.shape[0], -1) @ w1 + b1
hpreact = bngain*((hpreact - bnmean_running)/bnstd_running) + bnbias
h = torch.tanh(hpreact)
logits = h @ w2 + b2
print(logits.shape)
print(data_dict['dev'][1].shape)
loss = F.cross_entropy(logits, data_dict['dev'][1])
print(f"val loss after bn is : {loss}")

torch.Size([22633, 27])
torch.Size([22633])
val loss after bn is : 2.336761951446533


So we can see that the test yielded the same results as calculating the bnmean and bnstd again on the training set. This is the way it is implemented in the BatchNormalization layer of Pytorch.

#### Usually Batchnormalization Layer is placed after the layer which have multiplication operations, like Linear Layer or Convolutional Layer

#### Pytorchifying the code we have written so far becuase we are going to be using pytorch in practice.

In [5]:
g = torch.Generator().manual_seed(2147483647) # Declare the generator with the manual seed.

# So we would convert our code into modules like the Linear Module and  the BatchNorm Module and then we would write the training loop pytorch style
class Linear():
    """ This class creates a Linear layer with the weights and biases and performs the calculations"""
    def __init__(self, fan_in, fan_out, bias = True):
        self.weight = torch.randn((fan_in, fan_out), generator=g) / (fan_in**0.5)
        self.bias = torch.zeros(fan_out) if bias else None
    
    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out

    def parameters(self):
        return [self.weight, [] if self.bias is None else self.bias]

class BatchNorm1D():
    """ This class creates a Batch Normalization layer"""
    def __init__(self, dim, epsilon = 1e-5, momentum = 0.1):
        self.epsilon = epsilon
        self.momentum = momentum
        self.training = True
        # Initializing learnable paramters
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

        # Buffers trained with a momentum update
        self.running_mean = torch.zeros(dim)
        self.running_variance = torch.ones(dim)

    def __call__(self, x):
        # Calculate the forward pass
        if self.training:
            xmean = x.mean(0, keepdim=True)
            xvar = x.var(0,keepdim = True, unbiased = True)
        else:
            xmean = self.running_mean
            xvar = self.running_variance

        xhat = (x-xmean)/ torch.sqrt(xvar+self.epsilon)
        self.out = self.gamma * xhat + self.beta

        # Now update the running mean buffers
        if self.training:
            with torch.no_grad():
                self.running_mean = (1-self.momentum) * self.running_mean + self.momentum * xmean
                self.running_variance = (1-self.momentum)* self.running_variance + self.momentum * xvar
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

class Tanh():
    def __call__(self,x):
        self.out = torch.tanh(x)
        return self.out
    def parameters(self):
        return []

In [8]:
n_embed = 10
n_hidden = 100
block_size = 3
g = torch.Generator().manual_seed(2147483647) # Declare the generator with the manual seed.
c = torch.randn(len(chartoidx), n_embed)

layers = [
    Linear(n_embed*block_size, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, len(chartoidx))
]


with torch.no_grad():
    # Make the output softmax less confident
    layers[-1].weight *= 0.1

    for layer in layers[:-1]:
        # apply the gain parameters of the kaiming initialization
        if isinstance(layer, Linear):
            layer.weight *= 5/3
# calculate the parameters and make the parameters requires_grad attribute = True
parameter_list = [c] + [p for layer in layers for p in layer.parameters()]
# print(len(parameter_list))
# for x in parameter_list:
#     print(x.nelement())

p = sum([x.nelement() for x in parameter_list])
print(p)
for p in parameter_list:
    p.requires_grad = True

46497


In [9]:
# First we would need to train the Neural Net and then we would need to get the values activation for plotting
data_dict ={'train':[X_train, y_train], 
            'dev':[X_dev, y_dev],
            'test':[X_test, y_test]}

epochs = 10000
batch_size = 32
lossi = []
update_data_ratios = []
# Now we would write the training loop
def training_loop(epochs, batch_size, data, is_val_set = False):
    if is_val_set == False:
        for epoch in range(0,epochs+1):
            # get random 32 indices to take from the training set
            ix = torch.randint(0, data_dict['train'][0].shape[0], size=(32,), generator=g) # returns 32 random integers between 0 and len(X_train)
            data = data_dict['train'][0][ix] # This would get the actual training data of shape (32,3) i.e (batch_size, block_size) 
            embeddings = c[data]  # This will get the embeddings of the each of the data i.e (32,3,10)

            # Now we would convert the embedding tensor to a shape that will be used by the model

            x = embeddings.view(embeddings.shape[0],-1) # This is the reshaped data that would be passed into the model

            # Now we would get the activations of the layers by inputting the data into our model
            for layer in layers:
                x = layer(x)
            loss = F.cross_entropy(x, data_dict['train'][1][ix])

            # Now do the backward pass and retain the gradients
            for layer in layers:
                layer.out.retain_grad()
            # zero out the gradients
            for p in parameter_list:
                p.grad = None
            
            # Do the backward propagation
            loss.backward()

            # Update the parameters
            lr = 0.1 if i<50000 else 0.01
            for p in parameter_list:
                p.data += -lr*p.grad
            
            with torch.no_grad():
                update_data_ratios.append([(lr*p.grad.std()/p.data.std()).log10().item() for p in parameter_list])

            if epoch%1000 == 0:
                lossi.append(loss)
                print(f"epochs: {epoch:}/{epochs:} | loss :{loss.item():.4f}")
            if epoch == 1000:
                break
            
training_loop(epochs,batch_size, data_dict)

epochs: 0/10000 | loss :3.3290
epochs: 1000/10000 | loss :2.1367


In [10]:
update_data_ratios

[[-3.941789150238037,
  -3.3945260047912598,
  0.0,
  -3.2010738849639893,
  0.0,
  -3.2461633682250977,
  0.0,
  -3.2699830532073975,
  0.0,
  -3.3255648612976074,
  0.0,
  -0.6305078864097595,
  0.0],
 [-3.832125425338745,
  -3.3375630378723145,
  -0.17167365550994873,
  -3.1564269065856934,
  -0.1776381880044937,
  -3.1991727352142334,
  -0.16157621145248413,
  -3.2529492378234863,
  -0.1792270690202713,
  -3.3208136558532715,
  -0.22190037369728088,
  -0.6538206934928894,
  -0.2208443582057953],
 [-3.803764820098877,
  -3.287630081176758,
  -0.2706860303878784,
  -3.0874388217926025,
  -0.2603561282157898,
  -3.1544573307037354,
  -0.30300572514533997,
  -3.2003719806671143,
  -0.3410446345806122,
  -3.2788257598876953,
  -0.3839317560195923,
  -0.6503620743751526,
  -0.38154152035713196],
 [-3.7328758239746094,
  -3.20951509475708,
  -0.25353923439979553,
  -3.0272128582000732,
  -0.3125694990158081,
  -3.0742619037628174,
  -0.34251150488853455,
  -3.132842779159546,
  -0.4118919

Now as we have pytorchified the code. We might want to look deeper into the gradient distributions so that we can see how some factors affect our activations and gradients. So will need to plot layer by layer activations to see what the gradients and activations are behaving.

Now lets write the code for visualizing the actvations and the gradients of the tanh layer.
1. So we would look at the activations and the gradients of the tanh layer and plot the histograms of them with density plots and see how they behave

In [11]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

fig = go.Figure()
for i,layer in enumerate(layers[:-1]):
    if isinstance(layer, Tanh):
        l = f"Layer {i}"
        d = layer.out.detach()
        
        hy,hx = torch.histogram(d, density=True)
        fig.add_trace(go.Scatter(x = hx[:-1], y=hy, name = l))
        print(f"{l} | mean = {d.mean()} | std = {d.std()} | % Saturated = {(d.abs()>0.97).float().mean() * 100} %")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="center",
    x=0.50,
    title = 'Layers'
), title = "Tanh Activations")
fig.show()

Layer 1 | mean = 0.04296025261282921 | std = 0.7809546589851379 | % Saturated = 26.15625 %
Layer 3 | mean = -0.06338478624820709 | std = 0.7284252047538757 | % Saturated = 14.5625 %
Layer 5 | mean = 0.017545385286211967 | std = 0.720354437828064 | % Saturated = 12.78125 %
Layer 7 | mean = -0.0012873810483142734 | std = 0.7171831130981445 | % Saturated = 10.9375 %
Layer 9 | mean = -0.03727932274341583 | std = 0.7099339365959167 | % Saturated = 9.9375 %


We would need to see a similar graph of the gradients also around the tanh layer.

In [12]:
fig = go.Figure()
for i,layer in enumerate(layers[:-1]):
    if isinstance(layer, Tanh):
        l = f"Layer {i}"
        d = layer.out.grad.detach()
        print(f"{l} | mean = {d.mean()} | std = {d.std()}")
        hy,hx = torch.histogram(d, density=True)
        fig.add_trace(go.Scatter(x = hx[:-1], y=hy, name = l))

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title = 'Layers'
), title = "Tanh Layer Gradients")

fig.show()

Layer 1 | mean = -8.553550287615508e-06 | std = 0.0031257872469723225
Layer 3 | mean = 7.243812433443964e-05 | std = 0.0029941995162516832
Layer 5 | mean = -7.152633043006063e-05 | std = 0.0028540384955704212
Layer 7 | mean = -2.525479249015916e-05 | std = 0.002731181448325515
Layer 9 | mean = -5.494374272529967e-05 | std = 0.002294790931046009


Now we would plot and see the grad/paramter_data ratio. The gradient to the parameter_data shows us the ratio of gradients of the respective parameters and the values in those parameters. This would give us a sense in which how the parameters would get updated.

In [13]:
fig = go.Figure()

for i, p in enumerate(parameter_list):
    # Now get inly those parameters which are 2 Dimensional:
    if p.ndim == 2:
        # get the gradients on each of the parameters
        t = p.grad
        print(f" W{i} {tuple(p.shape)} | gradient_mean = {t.mean()} | gradient_std = {t.std()} | grad:data = {(t.std()/p.std())}")
        # lets plot the gradients distribution
        hy, hx = torch.histogram(t, density=True) # This function takes a tensor and return the histogram value of that with bins to plot it.
        fig.add_trace(go.Scatter(x = hx[:-1], y=hy, name = f"Weight{i}"))

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title = 'Layer Parameters'
), title = "Parameter Gradients")

fig.show()


 W0 (27, 10) | gradient_mean = 0.000357982877176255 | gradient_std = 0.011155378073453903 | grad:data = 0.01112582627683878
 W1 (30, 100) | gradient_mean = -5.2248298743506894e-05 | gradient_std = 0.010183264501392841 | grad:data = 0.0329529270529747
 W3 (100, 100) | gradient_mean = 0.00010583876428427175 | gradient_std = 0.007844330742955208 | grad:data = 0.04640824347734451
 W5 (100, 100) | gradient_mean = 4.805128992302343e-05 | gradient_std = 0.0070517538115382195 | grad:data = 0.04187857732176781
 W7 (100, 100) | gradient_mean = 2.3610165953869e-05 | gradient_std = 0.006501327268779278 | grad:data = 0.03857836499810219
 W9 (100, 100) | gradient_mean = 4.234553489368409e-05 | gradient_std = 0.0051607899367809296 | grad:data = 0.030507514253258705
 W11 (100, 27) | gradient_mean = -6.622738468742284e-11 | gradient_std = 0.021472282707691193 | grad:data = 0.3118651807308197


But the thing is that the grad to data ratio is not a very informative chart. A more informative chart would be the `update(learning_rate*gradient)` to data graph for each paramter because it would show us the ratio of the `update(learning_rate*gradient)` to model parameter data which would give us insights on how fast or slow our model is training by telling us how much magnitude relative to the parameter the optimizer is updating the parameter. Because of the value of the parameter is large and our update is very small then the optimizer would update that parameter with a very small amuount at every iteration. So to cause a sinificant change in the parameter values we would need to go through more iterations, hence the learning will be slow and vice-versa.

So we will look at the  learning_rate vs parameter plot

In [14]:
fig = go.Figure()
for i,p in enumerate(parameter_list):
    if p.ndim == 2:
        fig.add_trace(go.Scatter(y = [update_data_ratios[j][i] for j in range(len(update_data_ratios))], name=f"Param {i}"))
fig.add_trace(go.Scatter(y = [-3.0]*len(update_data_ratios), line_color="Black", marker=dict(
            color='LightSkyBlue',
            size=20,
            line=dict(
                color='MediumPurple',
                width=2
            ))))
fig.show()

So the ratio of update to paramters should not be too much above 1e-3 or -3 in log scale. If its below -3 then the parameters are not learning fast enough.

#### So in the above cells we did not add the batchnormalization layer and plotted the graphs but we had to precisely callibrate the layers weights with a 5/3 gain and the kaiming initialization and other things. Now we will add the BatchNormalization Layer and plot the graphs again and hopefully we would  not need to precisely callibrate the parameters and the BatchNormalization layer would take care of these sort of things.

In [17]:
n_embed = 10
n_hidden = 100
block_size = 3
g = torch.Generator().manual_seed(2147483647) # Declare the generator with the manual seed.
c = torch.randn(len(chartoidx), n_embed)

layers = [
    Linear(n_embed*block_size, n_hidden),BatchNorm1D(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden),BatchNorm1D(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden),BatchNorm1D(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden),BatchNorm1D(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden),BatchNorm1D(n_hidden), Tanh(),
    Linear(n_hidden, len(chartoidx)),BatchNorm1D(len(chartoidx))
]


with torch.no_grad():
    # Because the last layer is batchNorm we would not be making the softmax less confident. We would make gamma less confident
    layers[-1].gamma*= 0.1
    # Make the output softmax less confident
    # layers[-1].weight *= 0.1

    for layer in layers[:-1]:
        # apply the gain parameters of the kaiming initialization
        if isinstance(layer, Linear):
            layer.weight *= 5/3
# calculate the parameters and make the parameters requires_grad attribute = True
parameter_list = [c] + [p for layer in layers for p in layer.parameters()]

p = sum([x.nelement() for x in parameter_list])
print(p)
for p in parameter_list:
    p.requires_grad = True

47551


In [18]:
# First we would need to train the Neural Net and then we would need to get the values activation for plotting
data_dict ={'train':[X_train, y_train], 
            'dev':[X_dev, y_dev],
            'test':[X_test, y_test]}

epochs = 10000
batch_size = 32
lossi = []
update_data_ratios = []
# Now we would write the training loop
def training_loop(epochs, batch_size, data, is_val_set = False):
    if is_val_set == False:
        for epoch in range(0,epochs+1):
            # get random 32 indices to take from the training set
            ix = torch.randint(0, data_dict['train'][0].shape[0], size=(32,), generator=g) # returns 32 random integers between 0 and len(X_train)
            data = data_dict['train'][0][ix] # This would get the actual training data of shape (32,3) i.e (batch_size, block_size) 
            embeddings = c[data]  # This will get the embeddings of the each of the data i.e (32,3,10)

            # Now we would convert the embedding tensor to a shape that will be used by the model

            x = embeddings.view(embeddings.shape[0],-1) # This is the reshaped data that would be passed into the model

            # Now we would get the activations of the layers by inputting the data into our model
            for layer in layers:
                x = layer(x)
            loss = F.cross_entropy(x, data_dict['train'][1][ix])

            # Now do the backward pass and retain the gradients
            for layer in layers:
                layer.out.retain_grad()
            # zero out the gradients
            for p in parameter_list:
                p.grad = None
            
            # Do the backward propagation
            loss.backward()

            # Update the parameters
            lr = 0.1 if i<50000 else 0.01
            for p in parameter_list:
                p.data += -lr*p.grad
            
            with torch.no_grad():
                update_data_ratios.append([(lr*p.grad.std()/p.data.std()).log10().item() for p in parameter_list])

            if epoch%1000 == 0:
                lossi.append(loss)
                print(f"epochs: {epoch:}/{epochs:} | loss :{loss.item():.4f}")
            if epoch == 1000:
                break
            
training_loop(epochs,batch_size, data_dict)

epochs: 0/10000 | loss :3.3036
epochs: 1000/10000 | loss :2.1990


In [19]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

fig = go.Figure()
for i,layer in enumerate(layers[:-1]):
    if isinstance(layer, Tanh):
        l = f"Layer {i}"
        d = layer.out.detach()
        
        hy,hx = torch.histogram(d, density=True)
        fig.add_trace(go.Scatter(x = hx[:-1], y=hy, name = l))
        print(f"{l} | mean = {d.mean()} | std = {d.std()} | % Saturated = {(d.abs()>0.97).float().mean() * 100} %")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="center",
    x=0.50,
    title = 'Layers'
), title = "Tanh Activations")
fig.show()

Layer 2 | mean = -0.005784636829048395 | std = 0.6321737766265869 | % Saturated = 3.0 %
Layer 5 | mean = 0.002020086394622922 | std = 0.6407151818275452 | % Saturated = 2.40625 %
Layer 8 | mean = -0.0013384738704189658 | std = 0.6405412554740906 | % Saturated = 2.59375 %
Layer 11 | mean = -0.002865071874111891 | std = 0.6384755969047546 | % Saturated = 2.59375 %
Layer 14 | mean = 0.0003825464809779078 | std = 0.6403025388717651 | % Saturated = 2.3125 %


In [20]:
fig = go.Figure()
for i,layer in enumerate(layers[:-1]):
    if isinstance(layer, Tanh):
        l = f"Layer {i}"
        d = layer.out.grad.detach()
        print(f"{l} | mean = {d.mean()} | std = {d.std()}")
        hy,hx = torch.histogram(d, density=True)
        fig.add_trace(go.Scatter(x = hx[:-1], y=hy, name = l))

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title = 'Layers'
), title = "Tanh Layer Gradients")

fig.show()

Layer 2 | mean = -3.492459576745488e-12 | std = 0.003130658296868205
Layer 5 | mean = 9.313225537987968e-12 | std = 0.00288782618008554
Layer 8 | mean = 8.149072562579907e-12 | std = 0.0027907092589884996
Layer 11 | mean = 1.7753336994941193e-11 | std = 0.002634651493281126
Layer 14 | mean = -3.492459576745488e-12 | std = 0.002618232974782586


In [21]:
fig = go.Figure()

for i, p in enumerate(parameter_list):
    # Now get inly those parameters which are 2 Dimensional:
    if p.ndim == 2:
        # get the gradients on each of the parameters
        t = p.grad
        print(f" W{i} {tuple(p.shape)} | gradient_mean = {t.mean()} | gradient_std = {t.std()} | grad:data = {(t.std()/p.std())}")
        # lets plot the gradients distribution
        hy, hx = torch.histogram(t, density=True) # This function takes a tensor and return the histogram value of that with bins to plot it.
        fig.add_trace(go.Scatter(x = hx[:-1], y=hy, name = f"Weight{i}"))

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title = 'Layer Parameters'
), title = "Parameter Gradients")

fig.show()

 W0 (27, 10) | gradient_mean = -7.243620286923047e-11 | gradient_std = 0.007988221012055874 | grad:data = 0.008223970420658588
 W1 (30, 100) | gradient_mean = 5.832336319144815e-05 | gradient_std = 0.007825584150850773 | grad:data = 0.025363275781273842
 W5 (100, 100) | gradient_mean = 3.3019347029039636e-05 | gradient_std = 0.006038330961018801 | grad:data = 0.03587944433093071
 W9 (100, 100) | gradient_mean = -8.651550160720944e-05 | gradient_std = 0.005940805654972792 | grad:data = 0.03545543551445007
 W13 (100, 100) | gradient_mean = 8.01355236035306e-06 | gradient_std = 0.005483950488269329 | grad:data = 0.03269926831126213
 W17 (100, 100) | gradient_mean = 7.106746488716453e-05 | gradient_std = 0.005362147465348244 | grad:data = 0.03177136927843094
 W21 (100, 27) | gradient_mean = -0.00016204154235310853 | gradient_std = 0.010011415928602219 | grad:data = 0.06004560738801956


In [22]:
fig = go.Figure()
for i,p in enumerate(parameter_list):
    if p.ndim == 2:
        fig.add_trace(go.Scatter(y = [update_data_ratios[j][i] for j in range(len(update_data_ratios))], name=f"Param {i}"))
fig.add_trace(go.Scatter(y = [-3.0]*len(update_data_ratios), line_color="Black", marker=dict(
            color='LightSkyBlue',
            size=20,
            line=dict(
                color='MediumPurple',
                width=2
            ))))
fig.show()

So even if we change the gain we will see that the activations and the gradients do not change by very much and are much more robust. But the update to data ratio is affected  so we may have to tune the learning rate if we are changing the scale of the activation that are coming into the batchNorms.

So as we are using batchNormalization we can also not set the kaiming initialization and we will observe that the activation and gradients are very well behaved, but we would have to tune the learning rate.

The  way we pytorchified the code and made the API is is same to core pytorch API. So we actually could import pytorch.nn as nn and just add nn. before our layer names to run the models with pytorch and we would acchieve the same results

The API used here is identical to the API that pytorch uses.